In [2]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
#from __future__ import print_function
import lime
import lime.lime_tabular
import dill

In [3]:
df=pd.read_csv("datasetfinal.csv")
df=df.drop('Timestamp',axis = 1)
df=df.drop('Name', axis = 1)
df=df.drop('satisfied', axis = 1)
df=df.drop('12th Stream',axis = 1)
df=df.drop('10th',axis = 1)
df=df.drop('12th',axis = 1)
df=df.drop('interest',axis = 1)
df=df.dropna()
df.head(3)

,creative,writing,outdoorwork,future,groupwork,enjoymost,enjoyleast,clubmost,clubleast,projectliked,projectdisliked,noengineering,drawing,futurejob,Opted
0,I am somewhat creative,A bit apprehensive. I get overwhelmed with so ...,I love the outdoors and wish I could work outs...,Building things with moving parts,I occasionally like working with others,Maths,Social Studies,Environment club,Student council,Optimizing the Uber Pool routes,Maintenance of existing infrastructure,Business,I am not very good,Environment,EEE
1,I am somewhat creative,Excited! I can share my theories with the world,I love the outdoors and wish I could work outs...,Designing or building sensor based technology,I enjoy working with others,Maths,Physics,Environment club,Robotics club,Prototyping a musical instrument for children,Prototyping a Blood pressure monitor,Maths,"I am not the best, but I am not the worst",Health,ECE
2,I am somewhat creative,Excited! I can share my theories with the world,I love the outdoors and wish I could work outs...,Designing or building sensor based technology,I enjoy working with others,Maths,Language Arts,Robotics club,Art or design club,Programming a robot that can make you dinner,Prototyping a musical instrument for children,Computer Science,I am not very good,Automotive,EEE


In [4]:
le_create=LabelEncoder()
le_write=LabelEncoder()
le_out=LabelEncoder()
le_future=LabelEncoder()
le_group=LabelEncoder()
le_enjoymost=LabelEncoder()
le_enjoyleast=LabelEncoder()
le_clubmost=LabelEncoder()
le_clubleast=LabelEncoder()
le_projectliked=LabelEncoder()
le_projectdisliked=LabelEncoder()
le_eng=LabelEncoder()
le_job=LabelEncoder()
le_draw=LabelEncoder()
le_opted=LabelEncoder()
df["creative"]= le_create.fit_transform(df["creative"])
df["writing"]= le_write.fit_transform(df["writing"])
df["outdoorwork"]= le_out.fit_transform(df["outdoorwork"])
df["future"]= le_future.fit_transform(df["future"])
df["groupwork"]= le_group.fit_transform(df["groupwork"])
df["enjoymost"]= le_enjoymost.fit_transform(df["enjoymost"])
df["enjoyleast"]= le_enjoyleast.fit_transform(df["enjoyleast"])
df["clubmost"]= le_clubmost.fit_transform(df["clubmost"])
df["clubleast"]= le_clubleast.fit_transform(df["clubleast"])
df["projectliked"]= le_projectliked.fit_transform(df["projectliked"])
df["projectdisliked"]= le_projectdisliked.fit_transform(df["projectdisliked"])
df["noengineering"]= le_eng.fit_transform(df["noengineering"])
df["futurejob"]= le_job.fit_transform(df["futurejob"])
df["drawing"]= le_draw.fit_transform(df["drawing"])

In [5]:
df.head(3)

,creative,writing,outdoorwork,future,groupwork,enjoymost,enjoyleast,clubmost,clubleast,projectliked,projectdisliked,noengineering,drawing,futurejob,Opted
0,1,0,0,0,2,5,8,4,7,4,3,0,1,3,EEE
1,1,2,0,2,1,5,7,4,6,7,6,6,0,4,ECE
2,1,2,0,2,1,5,5,6,0,5,7,1,1,0,EEE


In [6]:
feature=df.drop('Opted',axis = 1)
target=df["Opted"]
X_train, X_test, y_train, y_test = train_test_split(feature,target, test_size = 0.3)

In [7]:
svc_model = SVC(C=100, kernel='poly', degree=3, gamma='auto', probability=True)
svc_model.fit(X_train, y_train)
prediction = svc_model.predict(X_test)

In [8]:
acc = metrics.accuracy_score(y_test, prediction)
f1=metrics.f1_score(y_test, prediction,average='micro')
print("Accuracy:",acc)
print("f1_score:",f1)
from sklearn.metrics import classification_report, confusion_matrix, precision_score
print("")

 
print(classification_report(y_test, prediction))

Accuracy: 0.9821428571428571
f1_score: 0.9821428571428571

              precision    recall  f1-score   support

         BME       0.98      1.00      0.99        56
       CIVIL       0.97      0.99      0.98       145
         CSE       0.99      0.99      0.99       143
         ECE       1.00      0.98      0.99       129
         EEE       0.97      0.99      0.98       101
        MECH       0.99      0.95      0.97       154

    accuracy                           0.98       728
   macro avg       0.98      0.98      0.98       728
weighted avg       0.98      0.98      0.98       728



In [9]:
predict_fn_rf = lambda x: svc_model.predict_proba(x).astype(float)
xt = feature.values
explainer = lime.lime_tabular.LimeTabularExplainer(xt,feature_names = X_train.columns,mode="classification",class_names=['BME','CIVIL','CSE','ECE','EEE','MECH'])

In [10]:
data = {"model": svc_model,"le_create":le_create,"exp":explainer,"le_write":le_write,"le_out":le_out, "le_future": le_future, "le_group": le_group,"le_enjoymost":le_enjoymost,"le_enjyleast":le_enjoyleast,"le_clubmost":le_clubmost,"le_clubleast":le_clubleast,"le_projectliked":le_projectliked,"le_projectdisliked":le_projectdisliked,"le_eng":le_eng,"le_job":le_job,"le_draw":le_draw}
with open('svc_model1.pkl', 'wb') as file:
    dill.dump(data, file)